# Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

Run elastic search image

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

``` 
```bash
curl localhost:9200
```

What's the version.build_hash value?

Output:

```json
{
  "name" : "69bf270d13e5",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "x95q-KD2Qgq-yB7VBR0opA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}
```

**Answer:** 42f05b9372a9a4a470db3b52817899b99a76ee73

# Getting the data

In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

In [6]:
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [11]:
from tqdm.auto import tqdm

/home/roman/.local/share/virtualenvs/llm-zoomcamp-6WKyO3I_/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████| 948/948 [00:07<00:00, 121.58it/s]


Which function do you use for adding your data to elastic?  
**Answer:** index

# Q3. Searching

Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05


Look at the _score field.

In [13]:
def elastic_search(query, n = 1):
    search_query = {
        "size": n,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                #"filter": {
                #    "term": {
                #        "course": "data-engineering-zoomcamp"
                #    }
                #}
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    return response

In [14]:
a = elastic_search("How do I execute a command in a running docker container?")
best_result = a['hits']['hits'][0]
print("""
Best score: {res[_score]}
Best result: {res[_source][text]}
""".format(res=best_result, _source='_source', text='text', _score='_score'))


Best score: 84.050095
Best result: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)



**Anser:** 84.05

# Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [20]:
def elastic_search_ml(query, n = 1):
    search_query = {
        "size": n,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    return response
def get_n_question(query, n):
    a = elastic_search_ml(query, n)
    last_result = a['hits']['hits'][n-1]
    print("""
    Question: {res[_source][question]}
    """.format(res=last_result, _source='_source', question='question', _score='_score'))

In [21]:
get_n_question("How do I execute a command in a running docker container?", 3)


    Question: How do I copy files from a different folder into docker container’s working directory?
    


**Answer**: How do I copy files from a different folder into docker container’s working directory?

# Q5. Building a prompt

In [22]:
def elastic_search_ml(query, n = 1):
    search_query = {
        "size": n,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [23]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [24]:
def rag1(query): 
    search_results = elastic_search_ml(query, 3)
    prompt = build_prompt(query, search_results)
    return prompt

In [25]:
prompt = rag1("How do I execute a command in a running docker container?")
len(prompt)

1463

What's the length of the resulting prompt? (use the len function)

* 962
* **1462**  (as closest)
* 1962
* 2462

**Answer**: 1462

# Q6. Tokens

In [26]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [27]:
import tiktoken

In [28]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [29]:
len(encoding.encode(prompt))

323

Use the encode function. How many tokens does our prompt have?

* 122
* 222
* **322**
* 422

  
Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

**Answer:** 322

# Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

In [34]:
from openai import OpenAI

clientO = OpenAI()

def llm_openai(prompt):
    response = clientO.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [38]:
response = llm_openai(prompt)
print(response)

To execute a command in a running Docker container, you can follow these steps:

1. First, find the container ID of the running container using the `docker ps` command:
    ```bash
    docker ps
    ```
2. Then, use the `docker exec` command to execute a command in that specific container. For example, to start a bash session in the container, you can use:
    ```bash
    docker exec -it <container-id> bash
    ```

Replace `<container-id>` with the actual container ID obtained from the `docker ps` output.


# Bonus: calculating the costs (ungraded)

In [37]:
input_tokens = len(encoding.encode(prompt))

In [39]:
output_tokens = len(encoding.encode(response))

In [41]:
input_price = 0.005 / 1000
output_price = 0.015 / 1000

In [42]:
price_per_1000_requests = 1000 * (input_tokens * input_price + output_tokens * output_price)

In [44]:
print(f"The price for 1000 request is {price_per_1000_requests:.2f} dollars")

The price for 1000 request is 3.40 dollars
